In [1]:
import open3d as o3d
from sys import argv, exit
import numpy as np
from scipy.spatial.transform import Rotation as R
import copy
import os


np.random.seed(42)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Generating Cube vertices

In [2]:
def getVertices():
    points = [[0, 8, 8], [0, 0, 8], [0, 0, 0], [0, 8, 0], [8, 8, 8], [8, 0, 8], [8, 0, 0], [8, 8, 0]]

    vertices = []

    for ele in points:
        if(ele is not None):
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
            sphere.paint_uniform_color([0.9, 0.2, 0])

            trans = np.identity(4)
            trans[0, 3] = ele[0]
            trans[1, 3] = ele[1]
            trans[2, 3] = ele[2]

            sphere.transform(trans)
            vertices.append(sphere)

    return vertices, points

In [3]:
vertices, points = getVertices()

Generating Robot positions

In [4]:
def getFrames():
    # posei = ( x, y, z, thetaZ(deg) )

    poses = [[-12, 0, 0, 0], [-10, -4, 0, 30], [-8, -8, 0, 60], [-4, -12, 0, 75], [0, -16, 0, 80]]

    frames = []

    for pose in poses:
        T = np.identity(4)
        T[0, 3], T[1, 3], T[2, 3] = pose[0], pose[1], pose[2]
        T[0:3, 0:3] = R.from_euler('z', pose[3], degrees=True).as_matrix()

        frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.2, origin=[0, 0, 0])
        frame.transform(T)
        frames.append(frame)

    return frames, poses

In [5]:
frames, poses = getFrames()

Visualizing Ground Truth robot positions and cube vertices

In [6]:
def visualizeData(vertices, frames):
    geometries = []
    geometries = geometries + vertices + frames

    o3d.visualization.draw_geometries(geometries)

In [7]:
visualizeData(vertices, frames)

Generating cube as a local point cloud viewed from five different robot's positions.  
Adding noise in measurement of cube vertices in local frame from robot's five different positions to simulate inaccuracy in depth calculation in real sensors.

In [8]:
def getLocalCubes(points, poses):
    # Returns local point cloud cubes

    points = np.array(points)
    poses = np.array(poses)

    nPoses, nPoints, pointDim = poses.shape[0], points.shape[0], points.shape[1]
    cubes = np.zeros((nPoses, nPoints, pointDim))

    for i, pose in enumerate(poses):
        cube = []

        T = np.identity(4)
        T[0, 3], T[1, 3], T[2, 3] = pose[0], pose[1], pose[2]
        T[0:3, 0:3] = R.from_euler('z', pose[3], degrees=True).as_matrix()

        for pt in np.hstack((points, np.ones((points.shape[0], 1)))):
            ptLocal = np.linalg.inv(T) @ pt.reshape(4, 1)

            cube.append(ptLocal.squeeze(1)[0:3])

        cubes[i] = np.asarray(cube)

    return cubes


def addNoiseCubes(cubes, noise=0):
    noisyCubes = np.zeros(cubes.shape)

    for i in range(cubes.shape[0]):
        noiseMat = np.random.normal(0, noise, cubes[i].size).reshape(cubes[i].shape)
        noisyCubes[i] = cubes[i] + noiseMat

    return noisyCubes

In [9]:
gtCubes = getLocalCubes(points, poses)
noisyCubesHigh = addNoiseCubes(gtCubes, noise=2.0)
noisyCubesLow = addNoiseCubes(gtCubes, noise=0.2)

## Task

Now from above, we have local observations of the 8 points of the point cloud from every pose. Now the question is, if we optimize these redundant measurements, will we get close to ground truth?

Applying ICP to calculate the relative transformations between robot poses for odometry constraints in Pose Graph Optimization.  
ICP is initialized using known correspondences, so we would get a close form solution in step.

In [12]:
def icpTransformations(cubes):
    # T1_2 : 2 wrt 1 

    P1 = cubes[0]
    P2 = cubes[1]
    P3 = cubes[2]
    P4 = cubes[3]
    P5 = cubes[4]

    pcd1, pcd2, pcd3, pcd4, pcd5 = (o3d.geometry.PointCloud(), o3d.geometry.PointCloud(), 
    o3d.geometry.PointCloud(), o3d.geometry.PointCloud(), o3d.geometry.PointCloud())

    pcd1.points = o3d.utility.Vector3dVector(P1)
    pcd2.points = o3d.utility.Vector3dVector(P2)
    pcd3.points = o3d.utility.Vector3dVector(P3)
    pcd4.points = o3d.utility.Vector3dVector(P4)
    pcd5.points = o3d.utility.Vector3dVector(P5)

    corr = np.array([(i, i) for i in range(8)]) 

    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()

    T1_2 = p2p.compute_transformation(pcd2, pcd1, o3d.utility.Vector2iVector(corr))
    T2_3 = p2p.compute_transformation(pcd3, pcd2, o3d.utility.Vector2iVector(corr))
    T3_4 = p2p.compute_transformation(pcd4, pcd3, o3d.utility.Vector2iVector(corr))
    T4_5 = p2p.compute_transformation(pcd5, pcd4, o3d.utility.Vector2iVector(corr))

    draw_registration_result(pcd2, pcd1, T1_2)

    trans = np.array([T1_2, T2_3, T3_4, T4_5])

    return trans


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(source_temp)
    vis.add_geometry(target_temp)
    vis.get_render_option().point_size = 15
    vis.run()
    vis.destroy_window()

Registering two point clouds using transformation obtained using ICP. 

In [13]:
trans = icpTransformations(noisyCubesHigh)
# Here by giving `noisyCubesHigh`, we are just saying our relative poses are very noisy. 
# Do not worry about how we are obtaining it (ICP), just think of it as some noisy odometry
# sensor giving consecutive relative poses. 

Visualizing the quality of ICP transformations by projecting all local point clouds to the first robot's frame.

In [14]:
def registerCubes(trans, cubes):
    # Registering noisy cubes in first frame

    cloud1 = getCloud(cubes[0], [0.9, 0.2, 0])
    cloud2 = getCloud(cubes[1], [0, 0.2, 0.9])
    cloud3 = getCloud(cubes[2], [0.2, 0.9, 0])
    cloud4 = getCloud(cubes[3], [0.5, 0, 0.95])
    cloud5 = getCloud(cubes[4], [0.9, 0.45, 0])

    T1_2 = trans[0]
    T2_3 = trans[1]
    T3_4 = trans[2]
    T4_5 = trans[3]

    cloud2 = [ele.transform(T1_2) for ele in cloud2]
    cloud3 = [ele.transform(T1_2 @ T2_3) for ele in cloud3]
    cloud4 = [ele.transform(T1_2 @ T2_3 @ T3_4) for ele in cloud4]
    cloud5 = [ele.transform(T1_2 @ T2_3 @ T3_4 @ T4_5) for ele in cloud5]

    geometries = cloud1 + cloud2 + cloud3 + cloud4 + cloud5

    o3d.visualization.draw_geometries(geometries)


def getCloud(cube, color):
    vertices = []

    for ele in cube:
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.15)
        sphere.paint_uniform_color(color)

        trans = np.identity(4)
        trans[0, 3] = ele[0]
        trans[1, 3] = ele[1]
        trans[2, 3] = ele[2]

        sphere.transform(trans)
        vertices.append(sphere)

    return vertices

Due to noise in the sensor measurements, vertices are not overlapping.

In [15]:
registerCubes(trans, noisyCubesLow)

Generating G2O file for pose graph optimization.  
Robot poses are the form of `VERTEX_SE3:QUAT`, because our landmarks are in 3D. Poses are caculated  using ICP.  
`VERTEX_SE3:QUAT i x y z q_x q_y q_z q_w`  
Odometry edges are of the form `EDGE_SE3:QUAT`.  
`EDGE_SE3:QUAT i j x y z q_x q_y q_z q_w info(x, y, z, theta_x, theta_y, theta_z)`  
Cube vertices are of the form of `VERTEX_SE3:QUAT`. Cube vertices are intialized with respect to first frame.  
Landmark edges are of the form of `EDGE_SE3:QUAT`. Each landmark edge connects a robot position with the 8 cube vertices. So, we have `5x8=40` landmark edges.   
Information matrix for odometry edges, $\omega_{odom}$ = `20 0 0 0 0 0 20 0 0 0 0 20 0 0 0 20 0 0 20 0 20`   
Information matrix for landmark to robot edges, $\omega_{landmark}$ = `40 0 0 0 0 0 40 0 0 0 0 40 0 0 0 0.000001 0 0 0.000001 0 0.000001`  
![landmark_edges.png](attachment:landmark_edges.png)

In [20]:
def writeRobotPose(trans, g2o):
    # Tw_1: 1 wrt w

    start = [-12, 0, 0, 0]

    Tw_1 = np.identity(4)
    Tw_1[0, 3], Tw_1[1, 3], Tw_1[2, 3] = start[0], start[1], start[2]
    Tw_1[0:3, 0:3] = R.from_euler('z', start[3], degrees=True).as_matrix()

    T1_2, T2_3, T3_4, T4_5 = trans[0], trans[1], trans[2], trans[3]

    Tw_2 = Tw_1 @ T1_2
    Tw_3 = Tw_2 @ T2_3
    Tw_4 = Tw_3 @ T3_4
    Tw_5 = Tw_4 @ T4_5

    pose1 = [Tw_1[0, 3], Tw_1[1, 3], Tw_1[2, 3]] + list(R.from_matrix(Tw_1[0:3, 0:3]).as_quat())
    pose2 = [Tw_2[0, 3], Tw_2[1, 3], Tw_2[2, 3]] + list(R.from_matrix(Tw_2[0:3, 0:3]).as_quat())
    pose3 = [Tw_3[0, 3], Tw_3[1, 3], Tw_3[2, 3]] + list(R.from_matrix(Tw_3[0:3, 0:3]).as_quat())
    pose4 = [Tw_4[0, 3], Tw_4[1, 3], Tw_4[2, 3]] + list(R.from_matrix(Tw_4[0:3, 0:3]).as_quat())
    pose5 = [Tw_5[0, 3], Tw_5[1, 3], Tw_5[2, 3]] + list(R.from_matrix(Tw_5[0:3, 0:3]).as_quat())

    posesRobot = [pose1, pose2, pose3, pose4, pose5]

    sp = ' '

    for i, (x, y, z, qx, qy, qz, qw) in enumerate(posesRobot):
        line = "VERTEX_SE3:QUAT " + str(i+1) + sp + str(x) + sp + str(y) + sp + str(z) + sp + str(qx) + sp + str(qy) + sp + str(qz) + sp + str(qw) + '\n'
        g2o.write(line)


def writeOdom(trans, g2o):	
    sp = ' '
    info = '20 0 0 0 0 0 20 0 0 0 0 20 0 0 0 20 0 0 20 0 20'

    for i, T in enumerate(trans):
        dx, dy, dz = T[0, 3], T[1, 3], T[2, 3]

        qx, qy, qz, qw = list(R.from_matrix(T[0:3, 0:3]).as_quat())

        line = "EDGE_SE3:QUAT " + str(i+1) + sp + str(i+2) + sp + str(dx) + sp + str(dy) + sp + str(dz) + sp + str(qx) + sp + str(qy) + sp + str(qz) + sp + str(qw) + sp +  info + '\n'

        g2o.write(line)


def writeCubeVertices(cubes, g2o):
    cube1 = cubes[0]

    start = [-12, 0, 0, 0]

    Tw_1 = np.identity(4)
    Tw_1[0, 3], Tw_1[1, 3], Tw_1[2, 3] = start[0], start[1], start[2]
    Tw_1[0:3, 0:3] = R.from_euler('z', start[3], degrees=True).as_matrix()

    cube = []

    for pt in np.hstack((cube1, np.ones((cube1.shape[0], 1)))):
        ptWorld = Tw_1 @ pt.reshape(4, 1)

        cube.append(ptWorld.squeeze(1)[0:3])

    quat = "0 0 0 1\n"
    sp = ' '

    for i, (x, y, z) in enumerate(cube):
        line = "VERTEX_SE3:QUAT " + str(i+6) + sp + str(x) + sp + str(y) + sp + str(z) + sp + quat

        g2o.write(line)


def writeLandmarkEdge(cubes, g2o):
    quat = "0 0 0 1"
    sp = ' '
    info = '40 0 0 0 0 0 40 0 0 0 0 40 0 0 0 0.000001 0 0 0.000001 0 0.000001' 

    for i, cube in enumerate(cubes):
        for j, (x, y, z) in enumerate(cube):
            line  = "EDGE_SE3:QUAT " + str(i+1) + sp + str(j+6) + sp + str(x) + sp + str(y) + sp + str(z) + sp + quat + sp +  info + '\n'

            g2o.write(line)


def writeG2o(trans, cubes):
    g2o = open("noise.g2o", 'w')

    g2o.write('# Robot poses\n\n')

    writeRobotPose(trans, g2o)

    g2o.write("\n # Cube vertices\n\n")

    writeCubeVertices(cubes, g2o)

    g2o.write('\n# Odometry edges\n\n')

    writeOdom(trans, g2o)

    g2o.write('\n# Landmark edges\n\n')

    writeLandmarkEdge(cubes, g2o)

    g2o.write("\nFIX 1\n")

    g2o.close()

In [21]:
writeG2o(trans, noisyCubesLow)

In [22]:
# Optimizing g2o file

In [23]:
def optimize():
    cmd = "g2o -robustKernel Cauchy -robustKernelWidth 1 -o {} -i 50 {} > /dev/null 2>&1".format(
        "opt.g2o", "noise.g2o")
    os.system(cmd)

In [24]:
optimize()

Reading optimized g2o file. And again registering all local point clouds to the first frame using optimized poses.

In [25]:
def readG2o(fileName):
    f = open(fileName, 'r')
    A = f.readlines()
    f.close()

    poses = []

    for line in A:
        if "VERTEX_SE3:QUAT" in line:
            (ver, ind, x, y, z, qx, qy, qz, qw, newline) = line.split(' ')

            if int(ind) <= 5:
                T = np.identity(4)
                T[0, 3], T[1, 3], T[2, 3] = x, y, z
                T[0:3, 0:3] = R.from_quat([qx, qy, qz, qw]).as_matrix()

                poses.append(T)

    poses = np.asarray(poses)

    return poses


def getRelativeEdge(poses):
    T1_2 = np.linalg.inv(poses[0]) @ poses[1]
    T2_3 = np.linalg.inv(poses[1]) @ poses[2]
    T3_4 = np.linalg.inv(poses[2]) @ poses[3]
    T4_5 = np.linalg.inv(poses[3]) @ poses[4]

    trans = np.array([T1_2, T2_3, T3_4, T4_5])

    return trans

In [26]:
optPoses = readG2o("opt.g2o")
optEdges = getRelativeEdge(optPoses)

In [27]:
registerCubes(optEdges, noisyCubesLow)
# Here we are just seeing if optimized poses are bringing the original landmark observations 
# in respective frames closer. However: (read "TODO3: Results Analysis" below )

### TODO1: Keep things simple
Rewrite the code above as follows:
1. We can replicate realistic scenarios as we progress and as need arises. For now, let us keep things _really_ simple (it's ok even if it doesn't make complete sense wrt realistic scenarious, we just want to see tangible results after optimization here): Instead of complicating the above with ICP-SLAM and all, just do the following: optimize for each of the following simple cases:
    1. HN-GPI, 0N-RPO, 0N-GLI, 0N-RLO: For **initializing** pose variables, simply add high noise directly to GT instead of using ICP-SLAM. Ensure High Noise (HN) for (Global) Pose variable initialization (GPI), 0 Noise (0N) for Relative Pose Observations (RPO) (basically the edges), 0 Noise for (Global) Landmark Initialization (basically the landmark variables) and (Relative) Landmark Observations (basically the landmarks observed in every local frame).
    2. HN-GPI, LN-RPO, 0N-GLI, 0N-RLO: Same as above but LN-RPO (very Low Noise) instead of 0N-RPO.
    3. HN-GPI, 0N-RPO, HN-GLI, 0N-RLO: Keep the **initialization** of landmark variables very noisy, just do it manually instead of initializing using say first pose. Ensure the landmark **observations** in every local frame has 0 noise first (and optimize it).
    4. HN-GPI, LN-RPO, HN-GLI, LN-RLO: same as 3. but less noise compared to initialization (and optimize it) instead of 0 noise. Notice "LN-RPO, .., 0N-RLO" instead of "0N-RPO, .., 0N-RLO".

Keep information values for both landmark and pose same in all cases.
        
Note that in 1. and 3. cases, our objective function can in theory reach a minimum of 0 value. 

### TODO2: Dataset
The current code is hardcoded for the 8 point cube dataset at some places. Rewrite it so that it works for slightly more denser dataset:

Make the dataset denser. Firstly, add more points to the cube (but in an orderly way, for example at midpoint of every edge). Add another geometry such as sphere. Now optimize it.

### TODO3: Results analysis 
One way to look at whether results have improved is as above (`registerCubes(optEdges, noisyCubesLow)`). However, a better way to do it would be:
1. Visualize GT landmarks, initial landmarks and optimized landmarks using 3 different colors.
2. Quantitatively, add a simple L2 loss function. Basically, compare between [GT - initial landmark positions] and [GT - optimized landmark positions]. 
3. Do the above two for poses also.

### Caveats

1. In this notebook, for easier understanding we treated landmark nodes similar to pose nodes (except that they don't have orientation) as you can see in the `noise.g2o` file. However, G2O of course offers better way to handle landmarks.
2. If the ICP-SLAM based formulation taught in class was confusing, note that the goal of this task is just to check if landmark based optimization is working given a noisy initialization. If you want to clear your confusion about ICP-SLAM lecture, read this page: https://www.notion.so/saishubodh/LS-Optimization-in-Vision-and-Robotics-A-Generic-Formulation-19d3172204bf43f988e80a4461e0e75d#b695f0f4cf5e4c4487a2586c7c313e2a